In [70]:
## import data   

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
## Algorithms which are we tried. 

import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression   
import xgboost as xgb
from xgboost.sklearn import XGBClassifier  
from xgboost import plot_importance

import joblib 
import pickle
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import cross_val_score, GridSearchCV
  

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score, precision_score, accuracy_score, roc_auc_score, recall_score, roc_curve
from sklearn.metrics import make_scorer, confusion_matrix, classification_report   

import warnings 
warnings.filterwarnings('ignore')

In [71]:
df = pd.read_csv('heppatitis.csv')
df.head()

,Age,Gender,Nausea,difficulty breathing,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,Headache,fever,cold,Jaundice,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive


In [72]:
replace_value = {'Yes':1,'No':0}

df['Nausea'] = df['Nausea'].map(replace_value)

In [73]:
df['difficulty breathing'] = df['difficulty breathing'].map(replace_value)
df['sudden weight loss'] = df['sudden weight loss'].map(replace_value)
df['weakness'] = df['weakness'].map(replace_value)
df['Polyphagia'] = df['Polyphagia'].map(replace_value)
df['Genital thrush'] = df['Genital thrush'].map(replace_value)
df['visual blurring'] = df['visual blurring'].map(replace_value)
df['Itching'] = df['Itching'].map(replace_value)
df['Irritability'] = df['Irritability'].map(replace_value)
df['delayed healing'] = df['delayed healing'].map(replace_value)
df['Headache'] = df['Headache'].map(replace_value)
df['fever'] = df['fever'].map(replace_value)
df['cold'] = df['cold'].map(replace_value)
df['Jaundice'] = df['Jaundice'].map(replace_value)

In [74]:
replace_value = {'Male':1,'Female':0}
df['Gender'] = df['Gender'].map(replace_value)

In [75]:
replace_value = {'Positive':1,'Negative':0}
df['class'] = df['class'].map(replace_value)

In [76]:
df['class'].value_counts()

1    320
0    200
Name: class, dtype: int64

In [77]:
df.head(10)

,Age,Gender,Nausea,difficulty breathing,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,Headache,fever,cold,Jaundice,class
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1
5,55,1,1,1,0,1,1,0,1,1,0,1,0,1,1,1,1
6,57,1,1,1,0,1,1,1,0,0,0,1,1,0,0,0,1
7,66,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1
8,67,1,1,1,0,1,1,1,0,1,1,0,1,1,0,1,1
9,70,1,0,1,1,1,1,0,1,1,1,0,0,0,1,0,1


In [78]:
X = df.drop('class', axis=1)
y = df['class']

In [79]:
X.head()

,Age,Gender,Nausea,difficulty breathing,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,Headache,fever,cold,Jaundice
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1


In [80]:
from sklearn.model_selection import train_test_split

# help(train_test_split)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [81]:
## hyper parameter tuning 
model_parameters = {'n_estimators': [10, 50, 100, 200, 500, 750, 1000], 
                    'max_depth': [3, 5, 10],
                    'min_samples_leaf': [np.random.randint(1,10)], 
                    'max_features': [None, 'sqrt', 'log2']}


model = GradientBoostingClassifier(random_state = 10)


gscv_GBM = GridSearchCV(estimator = model, 
                        param_grid = model_parameters, 
                        cv = 5, 
                        verbose = 1, 
                        n_jobs = -1,
                        scoring = 'roc_auc')

gscv_GBM.fit(X_train, y_train)

Fitting 5 folds for each of 63 candidates, totalling 315 fits


GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=10),
             n_jobs=-1,
             param_grid={'max_depth': [3, 5, 10],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [6],
                         'n_estimators': [10, 50, 100, 200, 500, 750, 1000]},
             scoring='roc_auc', verbose=1)

In [82]:
## finding differnet parameters

print('The best parameters are -', gscv_GBM.best_params_)

The best parameters are - {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 6, 'n_estimators': 500}


In [83]:
## fuitting model with differnt hyber parameters

final_mod_GBM = GradientBoostingClassifier(**gscv_GBM.best_params_)
final_mod_GBM.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=5, max_features='sqrt', min_samples_leaf=6,
                           n_estimators=500)

In [84]:
X_train.head(1)

,Age,Gender,Nausea,difficulty breathing,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,Headache,fever,cold,Jaundice
434,53,1,1,1,1,1,1,0,1,0,1,0,1,0,1,0


In [85]:
final_mod_GBM.predict([[53, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]])[0]

1

In [86]:
joblib.dump(final_mod_GBM, 'final_model.pkl')

['final_model.pkl']

In [87]:
train_pred = final_mod_GBM.predict(X_train)
test_pred = final_mod_GBM.predict(X_test)

In [88]:

print('Classification report for train data is : \n',
      classification_report(y_train, train_pred))
print('Classification report for test data is : \n',
      classification_report(y_test, test_pred))

Classification report for train data is : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       167
           1       1.00      1.00      1.00       249

    accuracy                           1.00       416
   macro avg       1.00      1.00      1.00       416
weighted avg       1.00      1.00      1.00       416

Classification report for test data is : 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99        33
           1       1.00      0.99      0.99        71

    accuracy                           0.99       104
   macro avg       0.99      0.99      0.99       104
weighted avg       0.99      0.99      0.99       104



In [89]:
log_reg = LogisticRegression(solver = 'liblinear')
log_reg.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

In [90]:

train_pred = log_reg.predict(X_train)
test_pred = log_reg.predict(X_test)

In [91]:
print('Classification report for train data is : \n',
      classification_report(y_train, train_pred))
print('Classification report for test data is : \n',
      classification_report(y_test, test_pred))

Classification report for train data is : 
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       167
           1       0.97      0.91      0.94       249

    accuracy                           0.93       416
   macro avg       0.92      0.93      0.93       416
weighted avg       0.93      0.93      0.93       416

Classification report for test data is : 
               precision    recall  f1-score   support

           0       0.91      0.88      0.89        33
           1       0.94      0.96      0.95        71

    accuracy                           0.93       104
   macro avg       0.93      0.92      0.92       104
weighted avg       0.93      0.93      0.93       104

